Three regime indicators:
Disturbance frequency (landscape-level)
Disturbance size (patch-level)
Disturbance severity (patch-level)

In [15]:
'''importing libraries'''
import os
import numpy as np
from pathlib import Path
import glob
import pandas
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# conus
from scipy import stats  # for sen's slope
import pymannkendall as mk


# define the study period 1988-2022
years = np.arange(1988, 2023)
agent_codes = [1, 2, 3, 4, 5, 6, 7]


In [19]:
'''load all the disturbance patch data from the directory'''
dir_rec = '/gpfs/sharedfs1/zhulab/Shi/ProjectCONUSDisturbanceAgent/Analysis/V08PostRelease/conus_disturbance_patch'
tiles = sorted(glob.glob(os.path.join(dir_rec, 'h*')))

distdata_patch = []
for tile in tiles:
    # print(f'Processing {tile}')
    tile = Path(tile).stem
    for yr in years:
        for cd in agent_codes:
            # directory of tile, h001v005_dist_patch_1985_3.csv
            # create file path by using the tile, year, and agent code
            filepath_rec = os.path.join(dir_rec, tile, f'{tile}_dist_patch_{yr}_{cd}.csv')
            # to check the file exists or not
            if os.path.exists(filepath_rec):
                distdata_patch.append(pandas.read_csv(filepath_rec))
distdata_patch = pandas.concat(distdata_patch)

'''clean up dataset and normalize severity into 1-4'''
def calculate_severity_vectorized(df):
    # Define the severity quarters for different agents
    severity_quarters = {
        'Logging': [0.09, 0.11, 0.14],
        'Construction': [0.12, 0.16, 0.20],
        'Agricultural change': [0.14, 0.18, 0.23],
        'Stress': [0.06, 0.08, 0.11],
        'Wind/geohazard': [0.08, 0.10, 0.13],
        'Water dynamic': [0.13, 0.17, 0.23],
        'Fire': [0.08, 0.10, 0.12]
    }
    # Create a DataFrame to hold severity thresholds
    thresholds = pandas.DataFrame.from_dict(severity_quarters, orient='index')
    # rename index into a new column named 'agent'
    thresholds = thresholds.reset_index().rename(columns={'index': 'agent'})
    # rename the columns
    thresholds.columns = ['agent', 'mag_threshold_25', 'mag_threshold_50', 'mag_threshold_75']


    # Merge with the input DataFrame on the 'agent' column
    # Merge the input DataFrame with the thresholds DataFrame on the 'agent' column
    df = df.merge(thresholds, on='agent', how='left')

    # Compute the severity based on the thresholds
    conditions = [
        (df['magnitude'] <= df['mag_threshold_25']),
        (df['magnitude'] > df['mag_threshold_25']) & (df['magnitude'] <= df['mag_threshold_50']),
        (df['magnitude'] > df['mag_threshold_50']) & (df['magnitude'] <= df['mag_threshold_75']),
        (df['magnitude'] > df['mag_threshold_75'])
    ]
    choices = [1, 2, 3, 4]

    df['severity'] = np.select(conditions, choices, default=0)
    
    # Drop the threshold columns and return only the necessary columns
    return df

# drop the duplicates for the cases where the disturbance patch is across the neighboring tiles, that was labeled as 1.
distdata_noncross   = distdata_patch[distdata_patch['across_tile'] == 0].copy()
distdata_cross_tile = distdata_patch[distdata_patch['across_tile'] == 1].copy().drop_duplicates()
# merge the two dataframes
distdata_patch = pandas.concat([distdata_noncross, distdata_cross_tile])
# replace the agent code with agent name
distdata_patch.agent[distdata_patch.agent == 1] = 'Logging'
distdata_patch.agent[distdata_patch.agent == 2] = 'Construction'
distdata_patch.agent[distdata_patch.agent == 7] = 'Agricultural change'
distdata_patch.agent[distdata_patch.agent == 3] = 'Stress'
distdata_patch.agent[distdata_patch.agent == 4] = 'Wind/geohazard'
distdata_patch.agent[distdata_patch.agent == 5] = 'Water dynamic'
distdata_patch.agent[distdata_patch.agent == 6] = 'Fire'

# normalize patch magnitude to patch severity, 1-4
distdata_patch['magnitude'] = distdata_patch['patch_magnitude_mean'] / 100
distdata_patch = calculate_severity_vectorized(distdata_patch)

# sampling to reduce the data size in order to speed up the plotting
distdata_patch_display = distdata_patch.sample(frac=0.01, random_state=42)

In [17]:
'''load landscape-level data'''

## disturbance frequency to show here
# create an avariable to store the informaiton of the disturbance
des       = '/gpfs/sharedfs1/zhulab/Shi/ProjectCONUSDisturbanceAgent/Analysis/V08PostRelease/hexagon_grid_50km_dist_regime_shift_patch_1988_2022'
data_list = sorted(glob.glob(os.path.join(des, 'grid*.csv')))

distdata_grid = []
for data_file in data_list:
    distdata_grid.append(pandas.read_csv(data_file))
distdata_grid = pandas.concat(distdata_grid) 
# replace agent name to display version
distdata_grid.agent[distdata_grid.agent == 'all'] = 'General'
distdata_grid.agent[distdata_grid.agent == 'forest_management'] = 'Logging'
distdata_grid.agent[distdata_grid.agent == 'construction'] = 'Construction'
distdata_grid.agent[distdata_grid.agent == 'agriculture_activity'] = 'Agricultural change'
distdata_grid.agent[distdata_grid.agent == 'stress'] = 'Stress'
distdata_grid.agent[distdata_grid.agent == 'natural_hazard'] = 'Wind/geohazard'
distdata_grid.agent[distdata_grid.agent == 'water_dynamic'] = 'Water dynamic'
distdata_grid.agent[distdata_grid.agent == 'fire'] = 'Fire'

# select the grids that happened disturbance
distdata_grid_tmp = distdata_grid[distdata_grid['measurement'] == 'dist_frequency']
grid_ids = distdata_grid_tmp[distdata_grid_tmp['MeanY8822'] > 0]['Id'].unique()
del distdata_grid_tmp
# select the grids that happened disturbance
distdata_grid = distdata_grid[distdata_grid['Id'].isin(grid_ids)]

/tmp/ipykernel_2348/1846205991.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distdata_grid.agent[distdata_grid.agent == 'all'] = 'General'
/tmp/ipykernel_2348/1846205991.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distdata_grid.agent[distdata_grid.agent == 'forest_management'] = 'Logging'
/tmp/ipykernel_2348/1846205991.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distdata_grid.agent[distdata_grid.agent == 'const

In [18]:
'''to get the regime inforamtion'''

regime_indicators = {'frequency', 'size', 'severity', 'magnitude'}
dist_agents       = {'General', 'Logging', 'Construction', 'Agricultural change', 'Stress', 'Wind/geohazard', 'Water dynamic', 'Fire'}
# define regime table to store the regime information
record_regime     = pandas.DataFrame(columns=['regime', 'agent', 'mean', 'min', 'q01', 'q25', 'q50', 'q75', 'q99', 'max', 'std'])
# iterate through the regime indicators
for regime in regime_indicators:
    for agt in dist_agents:
        # print(f'Processing {regime} for {agt}')
        # landscape grid -level frequency
        if regime == 'frequency':
            regime_name = f'dist_{regime}'
            # select the dist regime for the agent
            distdata_grid_regime_agent = distdata_grid[distdata_grid['measurement'] == regime_name]
            distdata_grid_regime_agent = distdata_grid_regime_agent[distdata_grid_regime_agent['agent'] == agt]
            # fill nan with zero for the grid
            distdata_grid_regime_agent = distdata_grid_regime_agent.fillna(0)
            # to get value list of the regime
            _value = distdata_grid_regime_agent['MeanY8822']*100 # patch's frequency to per km2 from per ha
        # patch-level size
        elif regime == 'size':
            if agt != 'General': # select the dist regime for the agent
                distdata_patch_regime_agent = distdata_patch[distdata_patch['agent'] == agt]
            else:
                distdata_patch_regime_agent = distdata_patch
            # to get value list of the regime
            _value = distdata_patch_regime_agent['area'] # patch's size in ha
        elif regime == 'severity':
            regime_name = f'severity'
            if agt != 'General': # select the dist regime for the agent
                distdata_patch_regime_agent = distdata_patch[distdata_patch['agent'] == agt]
            else:
                distdata_patch_regime_agent = distdata_patch
            # to get value list of the regime
            _value = distdata_patch_regime_agent['severity'] # patch's severity 1-4
        elif regime == 'magnitude':
            regime_name = f'magnitude'
            if agt != 'General': # select the dist regime for the agent
                distdata_patch_regime_agent = distdata_patch[distdata_patch['agent'] == agt]
            else:
                distdata_patch_regime_agent = distdata_patch
            # to get value list of the regime
            _value = distdata_patch_regime_agent['magnitude'] # patch's mean magnitude
        # to get the regime information for the agent
        _mean = _value.mean()
        _min = _value.min()
        _q01 = _value.quantile(0.01)
        _q25 = _value.quantile(0.25)
        _q50 = _value.quantile(0.50)
        _q75 = _value.quantile(0.75)
        _q99 = _value.quantile(0.99)
        _max = _value.max()
        _std = _value.std()
        # add the regime information to the record_regime by concatenating the dataframe
        record_regime = pandas.concat([record_regime, pandas.DataFrame({'regime': regime, 'agent': agt, 'mean': _mean, 'min': _min, 'q01': _q01, 'q25': _q25, 'q50': _q50, 'q75': _q75, 'q99': _q99, 'max': _max, 'std': _std}, index=[0])], ignore_index=True)

# sorted record_regime by the regime list, that is frequency, size, and severity
record_regime = record_regime.sort_values(by=['regime', 'agent'])

# print the record_regime
print(record_regime)

# save regime information to csv
record_regime.to_csv('regime_conus.csv', index=False)

/tmp/ipykernel_2348/3336675446.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  record_regime = pandas.concat([record_regime, pandas.DataFrame({'regime': regime, 'agent': agt, 'mean': _mean, 'min': _min, 'q01': _q01, 'q25': _q25, 'q50': _q50, 'q75': _q75, 'q99': _q99, 'max': _max, 'std': _std}, index=[0])], ignore_index=True)


       regime                agent       mean       min       q01       q25  \
1   frequency  Agricultural change   0.132261  0.000000  0.000000  0.009326   
2   frequency         Construction   0.051214  0.000000  0.000023  0.001931   
0   frequency                 Fire   0.005976  0.000000  0.000000  0.000000   
5   frequency              General   0.363035  0.000023  0.002769  0.114341   
6   frequency              Logging   0.088332  0.000000  0.000000  0.000457   
4   frequency               Stress   0.008316  0.000000  0.000000  0.000034   
7   frequency        Water dynamic   0.072941  0.000000  0.000057  0.005905   
3   frequency       Wind/geohazard   0.003996  0.000000  0.000000  0.000000   
9    severity  Agricultural change   2.528630  1.000000  1.000000  2.000000   
10   severity         Construction   2.500882  1.000000  1.000000  2.000000   
8    severity                 Fire   2.445641  1.000000  1.000000  2.000000   
13   severity              General   2.512250  1.000